In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
titanic = sns.load_dataset('titanic')
titanic.head()


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


Separando característica importantes e dividindo em treino e teste

In [49]:
feature_names = ['pclass', 'female', 'age', 'fare'] 
titanic['female'] = titanic['sex'].map({'male': 0, 'female': 1}) # convertendo sexo em variável numérica criando nova coluna female com 0 e 1 para masculino e feminino.
titanic.dropna(subset=feature_names, inplace=True)  # 891 para 714 linhas removendo linhas com valores NaN nas colunas em feature_names.
X = titanic[feature_names].to_numpy() # convertendo para numpy array
y = titanic['survived'].to_numpy() # convertendo para numpy array

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    random_state=123) # 75% treino, 25% teste

In [14]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,female
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,0
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,1
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,1
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,1
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,0


In [50]:
print('Tamanho de X_train: ', X_train.shape)
print('Tamanho de X_test: ', X_test.shape)
print('Tamanho de y_train: ', y_train.shape)
print('Tamanho de y_test: ', y_test.shape)

Tamanho de X_train:  (535, 4)
Tamanho de X_test:  (179, 4)
Tamanho de y_train:  (535,)
Tamanho de y_test:  (179,)


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### Classe nn.Module, nesse caso de classificação binaria, responsável por criar o modelo de rede neural com PyTorch, definindo a arquitetura da rede neural, função de ativação e método forward para a passagem dos dados pela rede.

In [51]:
class ClassBin(nn.Module):
    # Construtor
    def __init__(self):
        super(ClassBin, self).__init__()
        self.linear1 = nn.Linear(4, 4)    # primeira hidden layer
        self.dropout1 = nn.Dropout(0.2)   # dropout layer
        self.linear2 = nn.Linear(4, 4)    # segunda hidden layer
        self.dropout2 = nn.Dropout(0.2)   # dropout layer
        self.linear3 = nn.Linear(4, 1)    # terceira hidden layer
        self.dropout3 = nn.Dropout(0.2)   # dropout layer
        self.sigmoid = nn.Sigmoid()

    # Propagação (Feed Forward)
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = self.dropout1(x)
        x = F.relu(self.linear2(x))
        x = self.dropout2(x)
        x = F.relu(self.linear3(x))
        x = self.dropout3(x)
        x = self.sigmoid(x)
        return x


model = ClassBin()
print(model)

ClassBin(
  (linear1): Linear(in_features=4, out_features=4, bias=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (linear2): Linear(in_features=4, out_features=4, bias=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (linear3): Linear(in_features=4, out_features=1, bias=True)
  (dropout3): Dropout(p=0.2, inplace=False)
  (sigmoid): Sigmoid()
)


Loss function: Binary Cross Entropy Loss (BCELoss) - usada para problemas de classificação binária, mede a diferença entre as previsões do modelo e os rótulos reais. 
Otimizador: ajusta os pesos da rede neural com base no gradiente do erro, usando o algoritmo de otimização Adam (Adaptive Moment Estimation), que combina as vantagens do AdaGrad e do RMSProp, adaptando a taxa de aprendizado para cada parâmetro.

In [57]:
loss_fn = nn.BCELoss()
epochs = 100
batch_size = 4  # X_train 535 / 32 = 16.71 (então são 17 batches de 32)
learning_rate = 0.1

# Instânciar o Otimizador Adam
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Convertendo X e y para tensores do PyTorch porque o PyTorch trabalha com tensores, que são estruturas de dados semelhantes a arrays do NumPy, mas otimizadas para operações em GPU.

In [58]:
from torch.utils.data import TensorDataset, DataLoader

# Converter X e y para torch.Tensor
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)

# Um Dataset de Tensores - Array [X, y]
train = TensorDataset(X_train, y_train)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

test = TensorDataset(X_test, y_test)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)

Treinando o modelo

In [59]:
for t in range(epochs):
    model.train()  # Colocar o modelo em modo de treinamento (calcula os gradientes)

    # Batch Size
    for data in train_loader:
        # dar nome aos bois
        X = data[0]
        y = data[1]

        # Propagação (Feed Forward)
        y_pred = model(X)

        # Calcular erro usando a função-custo
        # y precisa virar um Tensor com tamanho (batch_size, 1)
        loss = loss_fn(y_pred, y.unsqueeze_(1))

        # Zera os gradientes antes da Retro-propagação (Backpropagation)
        model.zero_grad()

        # Retro-propagação (Backpropagation)
        loss.backward()

        # Atualização dos parâmetros
        optimizer.step()

    # Fim da Época
    print(f"""Época {t + 1},
          Custo Treino: {round(loss.item(), 3)}""")

Época 1,
          Custo Treino: 0.693
Época 2,
          Custo Treino: 0.693
Época 3,
          Custo Treino: 0.693
Época 4,
          Custo Treino: 0.693
Época 5,
          Custo Treino: 0.693
Época 6,
          Custo Treino: 0.693
Época 7,
          Custo Treino: 0.693
Época 8,
          Custo Treino: 0.693
Época 9,
          Custo Treino: 0.693
Época 10,
          Custo Treino: 0.693
Época 11,
          Custo Treino: 0.693
Época 12,
          Custo Treino: 0.693
Época 13,
          Custo Treino: 0.693
Época 14,
          Custo Treino: 0.693
Época 15,
          Custo Treino: 0.693
Época 16,
          Custo Treino: 0.693
Época 17,
          Custo Treino: 0.693
Época 18,
          Custo Treino: 0.693
Época 19,
          Custo Treino: 0.693
Época 20,
          Custo Treino: 0.693
Época 21,
          Custo Treino: 0.693
Época 22,
          Custo Treino: 0.693
Época 23,
          Custo Treino: 0.693
Época 24,
          Custo Treino: 0.693
Época 25,
          Custo Treino: 0.693
Época 26,

Medindo a acurácia do modelo

In [56]:
model.eval()  # coloca o modelo em modo de avaliação (sem calcular gradientes)

train_pred = model(X_train)
train_pred = train_pred.detach().apply_(lambda x: 1 if x > 0.5 else 0)
train_acc = torch.sum(train_pred.flatten() == y_train) / train_pred.size(0)

test_pred = model(X_test)
test_pred = test_pred.detach().apply_(lambda x: 1 if x > 0.5 else 0)
test_acc = torch.sum(test_pred.flatten() == y_test) / test_pred.size(0)

print(f"Acurácia de Treino: {train_acc}")
print('\n ---------------------------\n')
print(f"Acurácia de Teste: {test_acc}")

Acurácia de Treino: 0.590654194355011

 ---------------------------

Acurácia de Teste: 0.6033519506454468


## Modificação da ClassBin para responder o exercício 3 

## Modelo Novo - solicitado no EX3


In [60]:
class ClassBin2(nn.Module):

    def __init__(self):
        super(ClassBin2, self).__init__()

        # primeira hidden layer  com 4 neurônios de entrada e 16 neurônios de saída
        self.linear1 = nn.Linear(4, 16)
        # segunda hidden layer # segunda hidden layer
        self.linear2 = nn.Linear(16, 8)
        # terceira hidden layer om 8 neurônios de entrada e 1 neurônio de saída
        self.linear3 = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()
        self.ReLu = nn.ReLU()

    # Propagação (Feed Forward)
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = self.sigmoid(x)
        return x


model2 = ClassBin2()
print(model2)

ClassBin2(
  (linear1): Linear(in_features=4, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (ReLu): ReLU()
)


In [61]:
loss_fn = nn.BCELoss()
epochs = 100
batch_size = 32  # X_train 535 / 32 = 16.71 (então são 17 batches de 32)
learning_rate = 0.1

# Instânciar o Otimizador Adam
optimizer = torch.optim.Adam(model2.parameters(), lr=learning_rate)

In [62]:
for t in range(epochs):
    model2.train()  # Colocar o modelo em modo de treinamento (calcula os gradientes)

    # Batch Size
    for data in train_loader:
        # dar nome aos bois
        X = data[0]
        y = data[1]

        # Propagação (Feed Forward)
        y_pred = model2(X)

        # Calcular erro usando a função-custo
        # y precisa virar um Tensor com tamanho (batch_size, 1)
        loss = loss_fn(y_pred, y.unsqueeze_(1))

        # Zera os gradientes antes da Retro-propagação (Backpropagation)
        model2.zero_grad()

        # Retro-propagação (Backpropagation)
        loss.backward()

        # Atualização dos parâmetros
        optimizer.step()

    # Fim da Época
    print(f"""Época {t + 1},
          Custo Treino: {round(loss.item(), 3)}""")

Época 1,
          Custo Treino: 0.693
Época 2,
          Custo Treino: 0.693
Época 3,
          Custo Treino: 0.693
Época 4,
          Custo Treino: 0.693
Época 5,
          Custo Treino: 0.693
Época 6,
          Custo Treino: 0.693
Época 7,
          Custo Treino: 0.693
Época 8,
          Custo Treino: 0.693
Época 9,
          Custo Treino: 0.693
Época 10,
          Custo Treino: 0.693
Época 11,
          Custo Treino: 0.693
Época 12,
          Custo Treino: 0.693
Época 13,
          Custo Treino: 0.693
Época 14,
          Custo Treino: 0.693
Época 15,
          Custo Treino: 0.693
Época 16,
          Custo Treino: 0.693
Época 17,
          Custo Treino: 0.693
Época 18,
          Custo Treino: 0.693
Época 19,
          Custo Treino: 0.693
Época 20,
          Custo Treino: 0.693
Época 21,
          Custo Treino: 0.693
Época 22,
          Custo Treino: 0.693
Época 23,
          Custo Treino: 0.693
Época 24,
          Custo Treino: 0.693
Época 25,
          Custo Treino: 0.693
Época 26,

In [71]:
model.eval()  # coloca o modelo em modo de avaliação (sem calcular gradientes)

train_pred = model(X_train)
train_pred = train_pred.detach().apply_(lambda x: 1 if x > 0.5 else 0)
train_acc = torch.sum(train_pred.flatten() == y_train) / train_pred.size(0)

test_pred = model(X_test)
test_pred = test_pred.detach().apply_(lambda x: 1 if x > 0.5 else 0)
test_acc = torch.sum(test_pred.flatten() == y_test) / test_pred.size(0)
print("Modelo Clássico \n")
print(f"Acurácia de Treino: {train_acc}")
print('---------------------------\n')
print(f"Acurácia de Teste: {test_acc}")
print('---------------------------\n')

model2.eval()  # coloca o modelo2 em modo de avaliação (sem calcular gradientes)

train_pred = model2(X_train)
train_pred = train_pred.detach().apply_(lambda x: 1 if x > 0.5 else 0)
train_acc = torch.sum(train_pred.flatten() == y_train) / train_pred.size(0)

test_pred = model2(X_test)
test_pred = test_pred.detach().apply_(lambda x: 1 if x > 0.5 else 0)
test_acc = torch.sum(test_pred.flatten() == y_test) / test_pred.size(0)
print(f" Modelo 2 - EX3\n")
print(f"Acurácia de Treino: {train_acc}")
print('---------------------------\n')
print(f"Acurácia de Teste: {test_acc}")

Modelo Clássico 

Acurácia de Treino: 0.590654194355011
---------------------------

Acurácia de Teste: 0.6033519506454468
---------------------------

 Modelo 2 - EX3

Acurácia de Treino: 0.590654194355011
---------------------------

Acurácia de Teste: 0.6033519506454468
